In [ ]:
! pip install xgboost
! pip install pandas
! pip install scikit-learn
! pip install pyarrow

In [1]:
################################ Libraries ################################
from Input_Variables.read_vars import xgboost_regression_model_storage_location, \
                                      linear_regression_model_storage_location, \
                                      random_forest_regression_model_storage_location, \
                                      factorization_machines_regression_model_storage, \
                                      random_seed
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.scaling_pipeline import Feature_Transformations
from Model_Creation.regression_models import Create_Regression_Models
import os


################################ Read In Modules ################################
reading_data=Reading_Data()
feature_transformations=Feature_Transformations()
create_regression_models=Create_Regression_Models()


################################ Read In Data ################################
# Training Summary Stats Data
training_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_train_bool_updated'),x),
                                               os.listdir('/cephfs/summary_stats/all_train_bool_updated')))
training_files=[i for i in training_files if not ('.crc' in i or 'SUCCESS' in i)]


# Cross Validation Summary Stats Data
val_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_val_bool_updated'), x),
                                          os.listdir('/cephfs/summary_stats/all_val_bool_updated')))
val_files=[i for i in val_files if not ('.crc' in i or 'SUCCESS' in i)]


# Calling DataFrames
summary_stats_train=reading_data.read_in_all_summary_stats(file_list=training_files)
summary_stats_val=reading_data.read_in_all_summary_stats(file_list=val_files)


################################ Combine Train and Cross Validation ################################
df_train_val_combined=summary_stats_train.union(summary_stats_val)
df_train_val_combined.show(2)
print((df_train_val_combined.count(), len(df_train_val_combined.columns)))


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/31 09:01:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/31 09:01:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


23/05/31 09:02:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+-----+-----------------+----------------+-------------+-------------+------------------+------------------+------------------+------+-----+-----+--------------------+--------------------+------------------+-----------------+----------+----------+---------------+------------+------+----------+--------+------------------+-------------------------+------------+-------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|NumId|Chunk|ShortTermVariance|LongTermVariance|VarianceRatio|SampleEntropy|PermutationEntropy|              Mean|            StdDev|Median|  Min|  Max|        AvgFirstDiff|          AvgSecDiff|      StdFirstDiff|       StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|DiffPrevious|target|Sex_Female|Sex_Male|Treatment_yes_both|Treatment_yes_long_acting|Treatment_no|Treatment_yes_fast_acting|AgeGroup_50|AgeGroup_60|AgeGroup_70|AgeGroup_40|AgeGroup_30|AgeGroup_80|AgeGroup_90|AgeGroup_10|
+-----+-----+-------

In [2]:
df_train_val_combined.select('target').show(4)

+------+
|target|
+------+
|     0|
|     0|
|    -1|
|     0|
+------+
only showing top 4 rows



In [16]:
from pyspark.sql.functions import count, countDistinct
import pyspark.sql.functions as F

In [24]:
count_df=df_train_val_combined.groupBy("target").agg(count("target")).withColumnRenamed('count(target)', 'count')

In [25]:
count_df.show()

+------+-------+
|target|  count|
+------+-------+
|    -1|1136643|
|     1|1146444|
|     0|1337471|
+------+-------+



3620558

In [29]:
tot = count_df.agg(F.sum("count")).collect()[0][0]

In [34]:
df_train_val_combined.groupBy("target").agg(count("target")).withColumnRenamed('count(target)', 'count') \
  .withColumn('count', (F.col('count') / tot) * 100 ) \
  .show()

+------+------------------+
|target|             count|
+------+------------------+
|    -1|31.394138693538398|
|     1|31.664842822570442|
|     0|36.941018483891156|
+------+------------------+



In [ ]:
tot = count_df.count()

In [21]:
count_df.groupBy("target").agg(count("target"))\
  .withColumnRenamed('count(target)', 'cnt_per_group') \
  .withColumn('perc_of_count_total', (F.col('cnt_per_group') / tot) * 100 ) \
  .show()

+------+-------------+-------------------+
|target|cnt_per_group|perc_of_count_total|
+------+-------------+-------------------+
|    -1|            1|  33.33333333333333|
|     1|            1|  33.33333333333333|
|     0|            1|  33.33333333333333|
+------+-------------+-------------------+



In [ ]:
3,620,558